In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append(f'../../src')

import os
import glob
import util as ut 
import shutil
import dataset as ds
import nltk

## Helper Functions

In [4]:
def sets_counts(path):
    train_pos_size = len(glob.glob(f'{path}/dataset/train/pos/*.txt'))
    eval_pos_size  = len(glob.glob(f'{path}/dataset/eval/pos/*.txt'))
    test_pos_size  = len(glob.glob(f'{path}/dataset/test/pos/*.txt'))
    
    train_neg_size = len(glob.glob(f'{path}/dataset/train/neg/*.txt'))
    eval_neg_size  = len(glob.glob(f'{path}/dataset/eval/neg/*.txt'))
    test_neg_size  = len(glob.glob(f'{path}/dataset/test/neg/*.txt'))

    all_pos_size   = train_pos_size + eval_pos_size + test_pos_size
    all_neg_size   = train_neg_size + eval_neg_size + test_neg_size

    print(f'All: {(all_pos_size, all_neg_size)}, Train: {(train_pos_size, train_neg_size)}, Eval: {(eval_pos_size, eval_neg_size)}, Test: {(test_pos_size, test_neg_size)}')

    return (train_pos_size, eval_pos_size, test_pos_size)



def download_dataset(
    path          = '../../',
    train_percent = 0.7, 
    eval_percent  = 0.2, 
    test_percent  = 0.1
):
    if os.path.exists(f'{path}/dataset'):
        return sets_counts(path)

    !rm -rf {path}/aclImdb
    !rm -rf {path}/aclImdb_v1.tar.gz

    !cd {path}; wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

    !cd {path}; tar -xf aclImdb_v1.tar.gz
    
    !cd {path}; rm aclImdb_v1.tar.gz
        
    !cd {path}; mv aclImdb dataset
    
    !cd {path}/dataset; mv train all
    !cd {path}/dataset; mv test/pos/* all/pos/
    !cd {path}/dataset; mv test/neg/* all/neg/
    !cd {path}/dataset; rm -rf test

    pos_files = glob.glob(f'{path}/dataset/all/pos/*.txt')
    neg_files = glob.glob(f'{path}/dataset/all/neg/*.txt')
  
    random.shuffle(pos_files)
    random.shuffle(neg_files)

    ut.mkdir(f'{path}/dataset/train/pos')
    ut.mkdir(f'{path}/dataset/train/neg')
    
    ut.mkdir(f'{path}/dataset/eval/pos')
    ut.mkdir(f'{path}/dataset/eval/neg')
    
    ut.mkdir(f'{path}/dataset/test/pos')
    ut.mkdir(f'{path}/dataset/test/neg')
 
    train_size = int(len(pos_files) * train_percent)
    eval_size  = int(len(pos_files) * eval_percent)
    test_size  = int(len(pos_files) * test_percent)

    for f in pos_files[:train_size]: shutil.copy(f,  f'{path}/dataset/train/pos')
    for f in neg_files[:train_size]: shutil.copy(f,  f'{path}/dataset/train/neg')

    for f in pos_files[train_size:train_size+eval_size]: shutil.copy(f,  f'{path}/dataset/eval/pos')
    for f in neg_files[train_size:train_size+eval_size]: shutil.copy(f,  f'{path}/dataset/eval/neg')

    for f in pos_files[train_size+eval_size:train_size+eval_size+test_size]: shutil.copy(f,  f'{path}/dataset/test/pos')
    for f in neg_files[train_size+eval_size:train_size+eval_size+test_size]: shutil.copy(f,  f'{path}/dataset/test/neg')
    
    !cd {path}/dataset; rm -rf all

    return sets_counts(path)

## Build Dataset

In [5]:
download_dataset(
    train_percent = 0.07, 
    eval_percent  = 0.03, 
    test_percent  = 0.01
)

All: (2359, 2359), Train: (1502, 1502), Eval: (643, 643), Test: (214, 214)

(1502, 643, 214)